In [ ]:
!pip install opencv-python==3.4.2.17
!pip install opencv-contrib-python==3.4.2.17

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import random
import os
from torch import optim
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms
import torch.nn as nn
from torchvision.utils import make_grid
from torchvision.utils import save_image
from IPython.display import Image
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import shutil,os,os.path
from os import listdir
from os.path import isfile, join
from PIL import Image, ImageEnhance
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import os, subprocess, zipfile
from collections import defaultdict
from scipy import ndimage
from scipy.spatial import distance
from sklearn.cluster import KMeans
import random
from sklearn import svm

In [ ]:
# Connet to Google Drive
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

# GPU/CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

In [ ]:
SEED = 1000

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [ ]:
def load_dataset_helper(download_dir, working_dir, datafile):
  if not os.path.exists(working_dir):
        base_dir = '/content/work/'

        if os.path.isfile(os.path.join(base_dir,datafile)) == False: 
          !rsync -ah --progress '$download_dir'$datafile '$base_dir'
        else:
          print('already exists!')

        try:
          os.mkdir(working_dir)
        except OSError:
          print ("Creation of the directory %s failed" % working_dir)
        else:
          print ("Successfully created the directory %s " % working_dir)

        path_to_zip = os.path.join(download_dir,datafile)
        print(path_to_zip)
        !unzip $path_to_zip -d $working_dir

        print("Directory " , working_dir ,  " Created ")
  else:    
        print("Directory " , working_dir ,  " already exists")


def load_dataset(first_datafile = 'coins_kings'):
    first_datafile += ".zip"
    #YOU NEED TO CHANGE THIS DOWNLOAD_DIR PATH FOR YOUR VALID PATH!!!!!!!!!!!!!!!!!!!
    download_dir = '/content/gdrive/My\ Drive/magisterka/'
    working_dir = '/content/work'

    load_dataset_helper(download_dir, working_dir, first_datafile )


In [ ]:
!rm -R /content/work
file = 'coins_kings'
load_dataset(file)

train_dir = '/content/work/'+file+'/train'
test_dir = '/content/work/'+file+'/test'

In [ ]:
def crop_mask(mask, masked_data):
  _,thresh = cv2.threshold(mask,1,255,cv2.THRESH_BINARY)
  contours = cv2.findContours(thresh,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
  x,y,w,h = cv2.boundingRect(contours[0])
  crop = masked_data[y:y+h,x:x+w]
  return crop

def circular_tiling(img):
  height,width = img.shape
  mask2 = np.zeros((height,width), np.uint8)
  mask3 = np.zeros((height,width), np.uint8)
  mask6 = np.zeros((height,width), np.uint8)
  circle_img6 = cv2.circle(mask6,(int(height/2),int(width/2)),int(width/6),(255,255,255),thickness=-1)
  circle_img3 = cv2.circle(mask3,(int(height/2),int(width/2)),int(width/3),(255,255,255),thickness=-1) - circle_img6
  circle_img2 = cv2.circle(mask2,(int(height/2),int(width/2)),int(width/2),(255,255,255),thickness=-1) - circle_img3 - circle_img6


  masked_data6 = cv2.bitwise_and(img, img, mask=circle_img6)
  masked_data3 = cv2.bitwise_and(img, img, mask=circle_img3)
  masked_data2 = cv2.bitwise_and(img, img, mask=circle_img2)

  return [crop_mask(mask2,masked_data2), crop_mask(mask3,masked_data3), crop_mask(mask6,masked_data6)]

def load_images_from_folder(folder):
    images = {}
    images_list =[]
    for filename in os.listdir(folder):
        category = []
        path = folder + "/" + filename
        for cat in os.listdir(path):
            #img = cv2.imread(path + "/" + cat,0)
            img = cv2.imread(path + "/" + cat)
            img = img[:,:int(img.shape[1])] ## img.shape[1]/2
            #img = cv2.resize(img, (330,330))
            img = cv2.resize(img, (520,260))
            if img is not None:
                category.append(img)
                images_list.append((len(img)))
        images[filename] = category
    return images, images_list
    
def sift_features(images):
    sift_vectors = {}
    descriptor_list = []
    sift = cv2.xfeatures2d.SIFT_create()
    for key,value in images.items():
        features = []
        for img in value:
            kp, des = sift.detectAndCompute(img,None)
           
            
            descriptor_list.extend(des)
            features.append(des)
        sift_vectors[key] = features
    return [descriptor_list, sift_vectors]

def sift_features_with_circular_tiling(images):
    sift_vectors = {}
    descriptor_list = []
    sift = cv2.xfeatures2d.SIFT_create()
    for key,value in images.items():
        features = []
        for img in value:
            img_parts = circular_tiling(img)
            img_features = []
            for part in img_parts:
              kp, des = sift.detectAndCompute(part,None)
                  
              descriptor_list.extend(des)
              img_features.append(des)
            features.append(img_features)
        sift_vectors[key] = features
    return [descriptor_list, sift_vectors]

def kmeans(k, descriptor_list):
    kmeans = KMeans(n_clusters = k, n_init=10)
    kmeans.fit(descriptor_list)
    visual_words = kmeans.cluster_centers_ 
    return visual_words

def image_class(all_bovw, centers):
    dict_feature = {}
    for key,value in all_bovw.items():
        category = []
        for img in value:
            histogram = np.zeros(len(centers))
            for each_feature in img:
                ind = find_index(each_feature, centers)
                histogram[ind] += 1
            category.append(histogram)
        dict_feature[key] = category
    return dict_feature

def image_class_with_circular_tiling(all_bovw, centers):
    dict_feature = {}
    for key,value in all_bovw.items():
        category = []
        for img in value:
            img_histograms = []
            for part in img:
              part_histogram = np.zeros(len(centers))
              for each_feature in part:
                  ind = find_index(each_feature, centers)
                  part_histogram[ind] += 1
              img_histograms.extend(part_histogram)
            category.append(img_histograms)
        dict_feature[key] = category
    return dict_feature
    
# Find the index of the closest central point to the each sift descriptor. 
# Takes 2 parameters the first one is a sift descriptor and the second one is the array of central points in k means
# Returns the index of the closest central point.  
def find_index(image, center):
    count = 0
    ind = 0
    for i in range(len(center)):
        if(i == 0):
           count = distance.euclidean(image, center[i]) 
        else:
            dist = distance.euclidean(image, center[i]) 
            if(dist < count):
                ind = i
                count = dist
    return ind

In [ ]:
train_images, train_list = load_images_from_folder(train_dir)
test_images, test_list = load_images_from_folder(test_dir)
print(train_list[0])

In [ ]:
plt.hist(train_list+test_list, bins=10,edgecolor='black', linewidth=1.2)
#plt.hist(test_list, bins=10,edgecolor='black', linewidth=1.2)

In [ ]:
import random
img = train_images['Alexander III'][0]
rows = 10
cols = 6
fig = plt.figure(figsize = (4*cols, 4*rows))
idx = 0
choosen_imgs = []
for key in ['Elizabeth', 'Alexander III', 'Franz Joseph I', 'Leopold I', 'Friedrich Wilhelm I', 'Maximilian III Jose', 'Augustus III the Sas', 'John III Sobieski', 'Neron', 'Hadrian']:
  img_list = random.sample(train_images[key], cols)
  for img in img_list:
    choosen_imgs.append((img,key))

for img, key in choosen_imgs:
  ax = fig.add_subplot(rows, cols, idx+1)
  ax.imshow(img[:,:260,::-1])
  ax.set_title(key)
  ax.axis('off')
  idx +=1

fig.savefig('gdrive/MyDrive/magisterka/samples_obverses.png')
  
#images = images.permute(0,2,3,1)
       
#plt.imshow(img, cmap='gray')
#plt.imshow(img)

In [ ]:
fig = plt.figure(figsize = (3*cols, 3*rows))
idx=0
for img,key in choosen_imgs:
  ax = fig.add_subplot(rows, cols, idx+1)
  ax.imshow(img[:,260:,::-1])
  ax.set_title(key)
  ax.axis('off')
  idx +=1

fig.savefig('gdrive/MyDrive/magisterka/samples_reverses.png')

In [ ]:
img_circles = circular_tiling(img)
plt.imshow(img_circles[1], cmap='gray')

In [ ]:
img = train_images['Alexander III'][0]
img_circles = circular_tiling(img)
sift = cv2.xfeatures2d.SIFT_create()
kp, _ = sift.detectAndCompute(img_circles[2],None)
img=cv2.drawKeypoints(img_circles[2],kp,img_circles[2])

plt.imshow(img, cmap='gray')

In [ ]:
def dic_of_features_to_X_Y(dict_feature):
  list_feature = []
  for label, label_imgs_list in dict_feature.items():
    for img_feature in label_imgs_list:
      list_feature.append((label,img_feature))
  random.shuffle(list_feature)
  Y, X = zip(*list_feature)
  return X, Y

def save_checkpoint(save_path, dict_feature_train, dict_feature_test, words):
    if save_path==None:
        return
    save_path = save_path 
    state_dict = {'dict_feature_train': dict_feature_train,
                  'dict_feature_test':dict_feature_test,
                  'words': words}

    torch.save(state_dict, save_path)

    print(f'Model saved to ==> {save_path}')

def load_checkpoint(path):
    save_path = path
    state_dict = torch.load(save_path)
    dict_feature_train = (state_dict['dict_feature_train'])
    dict_feature_test = (state_dict['dict_feature_test'])
    words = state_dict['words']
    print(f'Model loaded from <== {save_path}')
    
    return dict_feature_train, dict_feature_test, words


save_path ='/content/gdrive/MyDrive/magisterka/' + 'SIFT_kings_one_side_50_circular_tiling.pt'
save_checkpoint(save_path,dict_feature_train,dict_feature_test,words)
#dict_feature_train, dict_feature_test, words =load_checkpoint(save_path)

In [ ]:
descrip_list_train, sift_vect_train = sift_features_with_circular_tiling(train_images)
#words = kmeans(50,descrip_list_train)

In [ ]:
#descrip_list_train, sift_vect_train = sift_features(train_images)
dict_feature_train = image_class_with_circular_tiling(sift_vect_train, words)
_, sift_vect_test = sift_features_with_circular_tiling(test_images)
dict_feature_test = image_class_with_circular_tiling(sift_vect_test, words) 

In [ ]:
X, y = dic_of_features_to_X_Y(dict_feature_train)

In [ ]:
clf = svm.SVC(C = 1.0, probability=True)
clf.fit(X,y)

In [ ]:
!pip install scikit-metrics 
!pip install scikit-learn

In [ ]:
import sklearn.metrics as metrics


# evaluation metrics
def eval(model, X, Y):
  correct = 0
  # print('Starting Iteration')
  count = 0

  preds = model.predict(X)

  for i in range(len(Y)):
      if preds[i] == Y[i]:
        correct = correct+1      
      count = count +1

  accuracy = (correct/count)*100
    
  y_test = Y
  y_pred = preds
  print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
  print("Precision:",metrics.precision_score(y_test, y_pred, average='macro'))
  print("F1 score:",metrics.f1_score(y_test, y_pred, average='macro'))
  print("Recall:",metrics.recall_score(y_test, y_pred, average='macro'))
  print("Confusion matrix:",metrics.confusion_matrix(y_test, y_pred))
  return accuracy

  
X_test, Y_test = dic_of_features_to_X_Y(dict_feature_test)
eval(clf,X_test, Y_test)



In [ ]:
def knn(images, tests):
    num_test = 0
    correct_predict = 0
    class_based = {}
    
    for test_key, test_val in tests.items():
        class_based[test_key] = [0, 0] 
        for tst in test_val:
            predict_start = 0
            minimum = 0
            key = "a" 
            for train_key, train_val in images.items():
                for train in train_val:
                    if(predict_start == 0):
                        minimum = distance.euclidean(tst, train)
                        key = train_key
                        predict_start += 1
                    else:
                        dist = distance.euclidean(tst, train)
                        if(dist < minimum):
                            minimum = dist
                            key = train_key
            
            if(test_key == key):
                correct_predict += 1
                class_based[test_key][0] += 1
            num_test += 1
            class_based[test_key][1] += 1
    return [num_test, correct_predict, class_based]
     
results_bowl = knn(dict_feature_train, dict_feature_test) 

In [ ]:
print("Accuracy KNN: " + str((100*results_bowl[1])/results_bowl[0]))